# Laboratorio: Principios de RDF y RDFs

Objetivos:

*	Ejercitar modelamiento básico mediante RDF Schema.
*	Realizar representaciones graficas de los modelos RDF.
*	Emplear herramientas de validación y conversión de documentos RDF.
*	Familiarizarse con el uso y definición de vocabulario, prefijos y URIs.

En este laboratorio vamos a considerar el siguiente enunciado corto:

**Samanta es una persona que le gusta leer libros.
A Samanta le gustan los libros de romance como "El amor en los tiempos de cólera".
El amor en los tiempos de cólera fue escrito por Gabriel García Márquez.**

Nuestro objetivo es modelar el enunciado y representar el conocimiento inherente usando RDF y RDFs. Para ello vamos a usar Python y la libreria rdflib. 




In [7]:
# Importación de librerías, no olviden ejecutar esta celda.
from rdflib import Graph, Literal, RDF, URIRef, Namespace

## Paso 1: Definición de espacio de nombres

Nuestro primer paso es definir los siguientes prefijos:

* @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
* @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
* @prefix uex: <http://www.uniandes.web.semantica.example.org/> .
* @prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> . 

Los últimos dos prefijos son uex y uexvocab son espacios de nombre propios de la ontología que vamos a construir. El primero lo usaremos pare definir instancias y el segundo para definición de nuestro vocabulario (i.e. clases, propiedades, etc).

In [8]:
from rdflib.namespace import RDF, RDFS

g = Graph() # Todo comienza con la definición de un grafo RDF
# RDFLib ya tiene definido los espacios de nombres mas conocidos, simplemente los debemos vincular a un prefijo
g.bind("rdf", RDF) # vincular un espacio de nombres proporcionado por RDFLib a un prefijo
g.bind("rdfs", RDFS) # vincular un espacio de nombres proporcionado por RDFLib a un prefijo

# Para un espacio de nombre no existente en RDFLib simplemente lo definimos usando el método Namespace y lo vinculamos
UEX = Namespace("http://www.uniandes.web.semantica.example.org/")
g.bind("uex", UEX)

#Veamos que prefijos hasta el momento tiene nuestro grafo
for ns_prefix, namespace in g.namespaces():
    print(ns_prefix,namespace)

owl http://www.w3.org/2002/07/owl#
rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
xml http://www.w3.org/XML/1998/namespace
uex http://www.uniandes.web.semantica.example.org/


**Pregunta 1 (10p):** Cree el espacio de nombres uexvocab (http://www.uniandes.web.semantica.ejemplo.org/voca#) y vinculelo al grafo RDF. Use como nombre de variable del namespace *UEXVOCAB* (notese la capitalización de las letras). 

In [9]:
# Por favor en esta celda conteste la Pregunta 1.

#Espacio de nombres UEXVOCAB
UEXVOCAB = Namespace("http://www.uniandes.web.semantica.ejemplo.org/voca#")
g.bind("uexvocab", UEXVOCAB)

# Prefijos y espacios de nombres para verificar
for ns_prefix, namespace in g.namespaces():
    print(ns_prefix, namespace)


owl http://www.w3.org/2002/07/owl#
rdf http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs http://www.w3.org/2000/01/rdf-schema#
xsd http://www.w3.org/2001/XMLSchema#
xml http://www.w3.org/XML/1998/namespace
uex http://www.uniandes.web.semantica.example.org/
uexvocab http://www.uniandes.web.semantica.ejemplo.org/voca#


In [10]:
## PRUEBAS OCULTAS PARA CALIFICACIÓN PREGUNTA 1

## Paso 2: Definición de clases

De nuestro enunciado podemos identificar las siguientes clases: Persona, Libro, Libros_de_Romance, y Autor. Vamos a proceder creando estas clases dentro de nuestro grafo, usando la definición de tipo rdfs:Class. Para cada clase debemos por lo tanto crear los siguientes triples:

* *uexvocab:Persona rdf:type rdfs:Class .*
* *uexvocab:Libro rdf:type rdfs:Class .*
* *uexvocab:Libros_de_Romance rdf:type rdfs:Class .*
* *uexvocab:Autor rdf:type rdfs:Class .*

In [11]:
## Con el método add podemos añadir nuevas triples al grafo
# Esta celda puede generar error si no definio correctamente el espacio de nombres UEXVOCAB
g.add((UEXVOCAB.Persona, RDF.type, RDFS.Class))
g.add((UEXVOCAB.Libro, RDF.type, RDFS.Class))
g.add((UEXVOCAB.Libros_de_Romance, RDF.type, RDFS.Class))

<Graph identifier=N2438c9ff1e0c4638996eb917dd122455 (<class 'rdflib.graph.Graph'>)>

In [12]:
## Serialicemos en formato TTL (turtle) para ver como va nuestro grafo
print(g.serialize(format='n3'))

@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> .

uexvocab:Libro a rdfs:Class .

uexvocab:Libros_de_Romance a rdfs:Class .

uexvocab:Persona a rdfs:Class .




**Pregunta 2 (10p):** Hace falta definir la clase autor. Defina el triple y añadalo al grafo.

In [15]:
# Añadir la clase Autor al grafo
g.add((UEXVOCAB.Autor, RDF.type, RDFS.Class))

# Serializar y mostrar el grafo en formato Turtle para revisar
print(g.serialize(format='turtle'))


@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> .

uexvocab:Autor a rdfs:Class .

uexvocab:Libro a rdfs:Class .

uexvocab:Libros_de_Romance a rdfs:Class .

uexvocab:Persona a rdfs:Class .




In [16]:
## PRUEBAS OCULTAS PARA CALIFICACIÓN PREGUNTA 2

## Paso 3: Definición de propiedades

Para la definición de propiedades es fundamental identificar los predicados (verbos) en el enunciado. Del enunciado se pueden identificar: lee, esUna, gusta, esAutorDe, y esCreadorDe. Se definiran por lo tanto los siguientes triples:

* *uexvocab:lee rdf:type rdf:Property .*
* *uexvocab:esUna rdf:type rdf:Property .*
* *uexvocab:gusta rdf:type rdf:Property .*
* *uexvocab:esAutorDe rdf:type rdf:Property .*
* *uexvocab:esCreadorDe rdf:type rdf:Property .*


In [ ]:
## Agregamos algunos de los triples de definición de propiedades al grafo.
g.add((UEXVOCAB.lee, RDF.type, RDF.Property ))
g.add((UEXVOCAB.esUna, RDF.type, RDF.Property ))
g.add((UEXVOCAB.gusta, RDF.type, RDF.Property ))
g.add((UEXVOCAB.esAutorDe, RDF.type, RDF.Property  ))


In [ ]:
## Serialicemos en formato TTL (turtle) para ver como va nuestro grafo
print(g.serialize(format='n3'))

**Pregunta 3 (10p):** Hace falta definir la propiedad esCreadorDe. Defina el triple y añadalo al grafo.

In [17]:
# Por favor en esta celda conteste la Pregunta 3.

# your code here

# Añadir la propiedad esCreadorDe al grafo
g.add((UEXVOCAB.esCreadorDe, RDF.type, RDF.Property))

# Serializar y mostrar el grafo en formato Turtle para revisar
print(g.serialize(format='turtle'))

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> .

uexvocab:Autor a rdfs:Class .

uexvocab:Libro a rdfs:Class .

uexvocab:Libros_de_Romance a rdfs:Class .

uexvocab:Persona a rdfs:Class .

uexvocab:esCreadorDe a rdf:Property .




In [18]:
## PRUEBAS OCULTAS PARA CALIFICACIÓN PREGUNTA 3

## Paso 4: Definición de jerarquía de propiedades/clases, asi como restricciones de dominio y rango.

Jerarquía de propiedades: 
* *uexvocab:esAutorDe rdfs:subPropertyOf uexvocab:esCreadorDe .* (subpropiedad)

Jerarquía de clases:
* *uexvocab:Libros_de_Romance rdfs:subClassOf uexvocab:Libro .* (subclase)
* *uexvocab:Autor rdfs:subClassOf uexvocab:Persona .*  (subclase)

Restricciones de dominio:
* *uexvocab:lee rdfs:domain uexvocab:Persona .* (La acción *lee* solo puede ser atribuida a una instancia tipo persona)
* *uexvocab: rdfs:domain uexvocab:Persona  .* (La acción leer *esCreadorDe* puede ser atribuida a una instancia tipo persona)
* *uexvocab:esAutorDe rdfs:range uexvocab:Libro .* (El objeto de la acción *esAutorDe* debe recaer sobre una instancia de tipo libro)


In [ ]:
## Agregamos algunos de los triples al grafo.
g.add((UEXVOCAB.esAutorDe, RDFS.subPropertyOf, UEXVOCAB.esCreadorDe ))
g.add((UEXVOCAB.Autor, RDFS.subClassOf, UEXVOCAB.Persona  ))
g.add((UEXVOCAB.lee, RDFS.domain, UEXVOCAB.Persona  ))
g.add((UEXVOCAB.esCreadorDe, RDFS.domain, UEXVOCAB.Persona  ))
g.add((UEXVOCAB.esAutorDe, RDFS.range, UEXVOCAB.Libro  ))

**Pregunta 4 (10p)**: Hace falta definir que la clase Libros_de_Romance es una subclase de Libro (*uexvocab:Libros_de_Romance rdfs:subClassOf uexvocab:Libro*). Defina el triple y añadalo al grafo.

In [19]:
# Por favor en esta celda conteste la Pregunta 4.

# your code here

# Añadir el triple que define Libros_de_Romance como subclase de Libro
g.add((UEXVOCAB.Libros_de_Romance, RDFS.subClassOf, UEXVOCAB.Libro))

# Serializar y mostrar el grafo en formato Turtle para revisar
print(g.serialize(format='turtle'))


@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> .

uexvocab:Autor a rdfs:Class .

uexvocab:Libro a rdfs:Class .

uexvocab:Libros_de_Romance a rdfs:Class ;
    rdfs:subClassOf uexvocab:Libro .

uexvocab:Persona a rdfs:Class .

uexvocab:esCreadorDe a rdf:Property .




In [20]:
## PRUEBAS OCULTAS PARA CALIFICACIÓN PREGUNTA 4

## Paso 5: Definición de instancias

Llegamos al paso final de nuestro modelamiento!!!, vamos a definir las instancias de nuestra ontología. Según nuestro enunciado debemos definir las instancias: 

* uex:Samanta
* uex:El_amor_en_los_tiempos_de_colera
* uex:Gabriel_Garcia_Marquez

Para la definición debemos especificar su pertenencia a alguna de nuestras clases, por lo tanto se deben definir los triples:


* *uex:Samanta rdf:type uexvocab:Persona .*
* *uex:El_amor_en_los_tiempos_de_colera rdf:type uexvocab:Libros_de_Romance .*
* *uex:Gabriel_Garcia_Marquez rdf:type uexvocab:Autor .*

In [21]:
## Agregamos los triples de definición de instancias al grafo.
g.add((UEX.Samanta, RDF.type, UEXVOCAB.Persona ))
g.add((UEX.El_amor_en_los_tiempos_de_colera, RDF.type, UEXVOCAB.Libros_de_Romance ))
g.add((UEX.Gabriel_Garcia_Marquez, RDF.type, UEXVOCAB.Autor  ))

<Graph identifier=N2438c9ff1e0c4638996eb917dd122455 (<class 'rdflib.graph.Graph'>)>

**Pregunta 5 (10p)**: Hace falta definir las relaciones entre las instancias a través de las propiedadesde nuestra ontología, esto es:

* *uex:Gabriel_Garcia_Marquez uexvocab:esAutorDe uex:El_amor_en_los_tiempos_de_colera .*  
* *uex:Samanta uexvocab:lee uex:El_amor_en_los_tiempos_de_colera*

Defina los triples y añadalos al grafo.

In [22]:
# Por favor en esta celda conteste la Pregunta 5.

# your code here

# Añadir los triples que definen las relaciones entre las instancias
g.add((UEX.Gabriel_Garcia_Marquez, UEXVOCAB.esAutorDe, UEX.El_amor_en_los_tiempos_de_colera))
g.add((UEX.Samanta, UEXVOCAB.lee, UEX.El_amor_en_los_tiempos_de_colera))

# Serializar y mostrar el grafo en formato Turtle para revisar
print(g.serialize(format='turtle'))

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix uex: <http://www.uniandes.web.semantica.example.org/> .
@prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> .

uexvocab:Autor a rdfs:Class .

uexvocab:Libro a rdfs:Class .

uexvocab:Libros_de_Romance a rdfs:Class ;
    rdfs:subClassOf uexvocab:Libro .

uexvocab:Persona a rdfs:Class .

uexvocab:esCreadorDe a rdf:Property .

uex:Gabriel_Garcia_Marquez a uexvocab:Autor ;
    uexvocab:esAutorDe uex:El_amor_en_los_tiempos_de_colera .

uex:Samanta a uexvocab:Persona ;
    uexvocab:lee uex:El_amor_en_los_tiempos_de_colera .

uex:El_amor_en_los_tiempos_de_colera a uexvocab:Libros_de_Romance .




In [23]:
## PRUEBAS OCULTAS PARA CALIFICACIÓN PREGUNTA 5

## Grafo en formato turtle y validadores

In [24]:
## Vamos a imprimir nuestro documento final en turtle.
print(g.serialize(format='n3'))

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix uex: <http://www.uniandes.web.semantica.example.org/> .
@prefix uexvocab: <http://www.uniandes.web.semantica.ejemplo.org/voca#> .

uexvocab:Autor a rdfs:Class .

uexvocab:Libro a rdfs:Class .

uexvocab:Libros_de_Romance a rdfs:Class ;
    rdfs:subClassOf uexvocab:Libro .

uexvocab:Persona a rdfs:Class .

uexvocab:esCreadorDe a rdf:Property .

uex:Gabriel_Garcia_Marquez a uexvocab:Autor ;
    uexvocab:esAutorDe uex:El_amor_en_los_tiempos_de_colera .

uex:Samanta a uexvocab:Persona ;
    uexvocab:lee uex:El_amor_en_los_tiempos_de_colera .

uex:El_amor_en_los_tiempos_de_colera a uexvocab:Libros_de_Romance .




### Validador

Existen muchos validadores RDF que nos permiten verificar la validez sintáctica de nuestro documento. El usar RDFlib ya es una garantía de la definición correcta triples, si embargo es bueno conocer herramientas externas como:

http://ttl.summerofcode.be/

Copie todas las líneas del documento turtle en la ventana del validador y presione el botón VALIDATE. Verá un texto que le confirma que la sintaxis del documento es correcta.

In [25]:
## En algunas ocacciones imprimir triple por tiple como strings nos permite encontrar errores de codificación y charsets.
## Para ellos simplemente debemos recorrer nuestro grafo.
print("--- printing raw triples ---")
for s, p, o in g:
    print((s, p, o))

--- printing raw triples ---
(rdflib.term.URIRef('http://www.uniandes.web.semantica.ejemplo.org/voca#Persona'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))
(rdflib.term.URIRef('http://www.uniandes.web.semantica.example.org/Gabriel_Garcia_Marquez'), rdflib.term.URIRef('http://www.uniandes.web.semantica.ejemplo.org/voca#esAutorDe'), rdflib.term.URIRef('http://www.uniandes.web.semantica.example.org/El_amor_en_los_tiempos_de_colera'))
(rdflib.term.URIRef('http://www.uniandes.web.semantica.ejemplo.org/voca#Autor'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))
(rdflib.term.URIRef('http://www.uniandes.web.semantica.ejemplo.org/voca#Libro'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#Class'))
(rdflib.term.URIRef('http://www.unia

## Felicitaciones ha construido su primera ontología!!

Puedes seguir enriqueciendo la ontología agregando nuevos triples!!